<a href="https://colab.research.google.com/github/cinnak/notion_edit/blob/main/notion_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from notion.client import NotionClient
import notion
import requests
import json
from google.colab import userdata
YOUR_API_KEY = userdata.get('your_key_name')
DATABASE_ID= "your_database_id"
headers = {
    "Authorization": "Bearer " + YOUR_API_KEY,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}


def get_pages(db_id,num_pages=None):
    """
    If num_pages is None, get all pages, otherwise just the defined number.
    """
    url = f"https://api.notion.com/v1/databases/{db_id}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    payload = {"page_size": page_size}
    response = requests.post(url, json=payload, headers=headers)

    data = response.json()
    results = data["results"]
    while data["has_more"] and get_all:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{db_id}/query"
        response = requests.post(url, json=payload, headers=headers)
        data = response.json()
        results.extend(data["results"])
    return results


def create_block(token, page_id, text_content):
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    data = {
	"children": [
		{
			"object": "block",
			"type": "paragraph",
			"paragraph": {
				"rich_text": [
					{
						"type": "text",
						"text": {
							"content": text_content
						}
					}
				]
			}
		}
	]
}
    response = requests.patch(url, headers=headers, data=json.dumps(data))
    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code} {response.text}")

if __name__=="__main__":
    pages = get_pages(DATABASE_ID,num_pages=None)
    for page in pages:
        content = page['properties']['简评']['rich_text'][0]['text']['content']
        page_id = page['id']
        create_block(YOUR_API_KEY, page_id, content)